# Example solution

## Loading the functions

In [1]:
import numpy as np
import pandas as pd

def calculate_eigen(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_eigenvalue = np.max(eigenvalues)
    max_eigenvector = eigenvectors[:, np.argmax(eigenvalues)]

    # Normalize the eigenvector to get the weights
    normalized_weights = max_eigenvector / np.sum(max_eigenvector)
    
    # Calculate the Consistency Index (CI)
    n = matrix.shape[0]
    CI = (max_eigenvalue - n) / (n - 1)
    
    # Random Consistency Index (RI), values depend on matrix size
    RI_dict = {1: 0, 2: 0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45,
           10: 1.49, 11: 1.52, 12: 1.54, 13: 1.56, 14: 1.58, 15: 1.59, 16: 1.60, 17: 1.61,
           18: 1.62, 19: 1.63, 20: 1.64, 21: 1.65, 22: 1.66, 23: 1.67, 24: 1.68, 25: 1.69,
           26: 1.70, 27: 1.71, 28: 1.72, 29: 1.73, 30: 1.74}
    RI = RI_dict.get(n, 1.49)  # 1.49 is an average fallback value
    
    # Calculate the Consistency Ratio (CR)
    CR = CI / RI
    
    consistency_interpretation = ("Consistent because CR is lower than 0.1") if CR <= 0.1 else "Inconsistent because CR is greater than CR"
    
    return max_eigenvalue, normalized_weights.real, CR, consistency_interpretation

def initialize_ahp_matrix(df, column_name):
    categories = df[column_name].tolist()
    n = len(categories)
    
    # Initialize a zero matrix of dimensions n x n
    ahp_matrix = np.zeros((n, n))
    
    # Create a labeled DataFrame to hold the AHP matrix
    ahp_df = pd.DataFrame(ahp_matrix, index=categories, columns=categories)
    
    return ahp_df

def generate_saaty_scale_with_explanations():
    return {
        'Equal Importance': 1,
        'Moderate Importance': 3,
        'Strong Importance': 5,
        'Very Strong Importance': 7,
        'Extreme Importance': 9,
        'Moderately Less Important': 1/3,
        'Strongly Less Important': 1/5,
        'Very Strongly Less Important': 1/7,
        'Extremely Less Important': 1/9
    }

def fill_ahp_matrix(ahp_df, row_name, col_names, comparison):
    saaty_scale = generate_saaty_scale_with_explanations()
    if comparison in saaty_scale:
        value = saaty_scale[comparison]
        for col_name in col_names:
            ahp_df.loc[row_name, col_name] = value
            ahp_df.loc[col_name, row_name] = 1 / value
    else:
        print("Invalid comparison description. Please select one from Saaty's scale.")
    return ahp_df

def populate_ahp_matrix(ahp_df):
    saaty_scale_dict = {i+1: option for i, option in enumerate(generate_saaty_scale_with_explanations().keys())}
    
    for row in ahp_df.index:
        temp_saaty_scale_dict = saaty_scale_dict.copy()
        
        criteria_dict = {i+1: col for i, col in enumerate(ahp_df.columns) if col != row and ahp_df.loc[row, col] == 0}
        temp_criteria_dict = criteria_dict.copy()
        
        while temp_criteria_dict:
            print(f"\nSelect an option for comparisons involving {row} against remaining criteria:")
            
            # Show available Saaty's scale options
            for num, option in temp_saaty_scale_dict.items():
                print(f"Saaty {num}. {option}")

            # Show remaining criteria mapped to numbers
            for num, criteria in temp_criteria_dict.items():
                print(f"Criteria {num}. {criteria}")

            saaty_selection = int(input("Enter the number of your Saaty scale selection: "))
            selected_comparison = temp_saaty_scale_dict[saaty_selection]

            print(f"Indicate all criteria from the list above that have '{selected_comparison}' when compared to {row}. Separate multiple criteria by comma.")
            relevant_cols_numbers = input().split(',')
            relevant_cols = [temp_criteria_dict[int(num.strip())] for num in relevant_cols_numbers]
            
            ahp_df = fill_ahp_matrix(ahp_df, row, relevant_cols, selected_comparison)

            # Pre-fill for transitive relations, i.e., if A = B and A = C, then B = C
            if selected_comparison == 'Equal Importance':
                for i in range(len(relevant_cols)):
                    for j in range(i+1, len(relevant_cols)):
                        ahp_df.loc[relevant_cols[i], relevant_cols[j]] = 1
                        ahp_df.loc[relevant_cols[j], relevant_cols[i]] = 1
            
            # Update temp_criteria_dict to remove selected items
            temp_criteria_dict = {num: col for num, col in temp_criteria_dict.items() if col not in relevant_cols}

            # Update temp_saaty_scale_dict to exclude the selected comparison
            del temp_saaty_scale_dict[saaty_selection]
    
    # Set diagonal elements to 1
    np.fill_diagonal(ahp_df.values, 1)
    
    return ahp_df


## Solving the problem

**In this example, we have the following assumptions:**

1.  "Visibility of System Status" is of equal importance to "Match Between System and Real World" and "User Control and Freedom."

2.  "Visibility of System Status" is moderately less important than "Consistency and Standards," "Error Prevention," and "Recognition Rather than Recall."

3.  "Match Between System and Real World" is moderately less important than "Consistency and Standards," "Error Prevention," and "Recognition Rather than Recall."

4.  "User Control and Freedom" is of equal importance to "Consistency and Standards."

5.  "User Control and Freedom" is moderately less important than "Error Prevention" and "Recognition Rather than Recall."

6.  "Consistency and Standards" is of equal importance to both "Error Prevention" and "Recognition Rather than Recall."

In [7]:
# Sample usage
data = {'Categories': ['Criteria_1', 'Criteria_2', 'Criteria_3', 'Criteria_4', 'Criteria_5', 'Criteria_6']}
df = pd.DataFrame(data)
ahp_df = initialize_ahp_matrix(df, 'Categories')

ahp_df = populate_ahp_matrix(ahp_df)

print(ahp_df)
max_eigenvalue, normalized_weights, CR, consistency_interpretation = calculate_eigen(ahp_df)
print("Max Eigenvalue:", max_eigenvalue)
print("Normalized Weights:", normalized_weights)
print("Consistency Ratio :", CR)
print("consistency interpretation :", consistency_interpretation)


Select an option for comparisons involving Criteria_1 against remaining criteria:
Saaty 1. Equal Importance
Saaty 2. Moderate Importance
Saaty 3. Strong Importance
Saaty 4. Very Strong Importance
Saaty 5. Extreme Importance
Saaty 6. Moderately Less Important
Saaty 7. Strongly Less Important
Saaty 8. Very Strongly Less Important
Saaty 9. Extremely Less Important
Criteria 2. Criteria_2
Criteria 3. Criteria_3
Criteria 4. Criteria_4
Criteria 5. Criteria_5
Criteria 6. Criteria_6
Indicate all criteria from the list above that have 'Equal Importance' when compared to Criteria_1. Separate multiple criteria by comma.

Select an option for comparisons involving Criteria_1 against remaining criteria:
Saaty 2. Moderate Importance
Saaty 3. Strong Importance
Saaty 4. Very Strong Importance
Saaty 5. Extreme Importance
Saaty 6. Moderately Less Important
Saaty 7. Strongly Less Important
Saaty 8. Very Strongly Less Important
Saaty 9. Extremely Less Important
Criteria 4. Criteria_4
Criteria 5. Criteria_

The output interpretation:

1. **Max Eigenvalue (λmax): 6.1414**: This value helps evaluate the consistency of the pairwise comparison matrix. Typically, for a perfectly consistent matrix, λmax would equal the number of criteria. In this case, the λmax value slightly exceeds the number of criteria (6), indicating a reasonable level of consistency.

2. **Normalized Weights**: These weights represent the relative importance of each criterion in the decision-making process. The values are as follows:
1. "Visibility of System Status": 8.43%
2. "Match Between System and Real World": 8.43%
3. "User Control and Freedom": 10.79%
4. "Consistency and Standards": 21.77%
5. "Error Prevention": 25.29%
6. "Recognition Rather than Recall": 25.29%
  
  These weights suggest that "Error Prevention" and "Recognition Rather than Recall" are the most important criteria, weighing approximately 25.29%. Then, "Consistency and Standards" at around 21.77%. The least important criteria are "Visibility of System Status" and "Match Between System and Real World," weighing approximately 8.43%.

3. **Consistency Ratio (CR): 0.0228**: This metric evaluates the reliability of the judgments. It is less than 0.1, which generally indicates a satisfactory level of consistency according to Saaty's guidelines.

4. **Consistency Interpretation**: The consistency ratio is less than 0.1; therefore, the pairwise comparisons are considered consistent. Consistent means that the judgments made in evaluating the criteria are reliable and can be used for decision-making because there are not random assignations or contradictory comparisons.

## Updating the usability score

We select six heuristics for this example and fix a "usability score":

1. **Visibility of System Status**: Score - 65
2. **Match Between System and Real World**: Score - 70
3. **User Control and Freedom**: Score - 75
4. **Consistency and Standards**: Score - 80
5. **Error Prevention**: Score - 82
6. **Recognition Rather than Recall**: Score - 85

Now, each heuristic has a weight as follows:
1. **Visibility of System Status**: Weight - 8.43%
2. **Match Between System and Real World**: Weight - 8.43%
3. **User Control and Freedom**: Weight - 10.79%
4. **Consistency and Standards**: Weight - 21.77%
5. **Error Prevention**: Weight - 25.29%
6. **Recognition Rather than Recall**: Weight - 25.29%


In [10]:
data = {'Categories': ['Visibility of System Status', 'Match Between System and Real World', 
                       'User Control and Freedom', 'Consistency and Standards', 'Error Prevention', 
                       'Recognition Rather than Recall'],
        'Usability_Score': [65,70,75,80,82,85], 
        'Weights' : [0.0843,0.0843,0.1079,0.2177,0.2529,0.2529]}

df = pd.DataFrame(data)
df

,Categories,Usability_Score,Weights
0,Visibility of System Status,65,0.0843
1,Match Between System and Real World,70,0.0843
2,User Control and Freedom,75,0.1079
3,Consistency and Standards,80,0.2177
4,Error Prevention,82,0.2529
5,Recognition Rather than Recall,85,0.2529


The relative weight is

In [12]:
df['RelativeWeight'] = df.Usability_Score*df.Weights
df

,Categories,Usability_Score,Weights,RelativeWeight
0,Visibility of System Status,65,0.0843,5.4795
1,Match Between System and Real World,70,0.0843,5.9010
2,User Control and Freedom,75,0.1079,8.0925
3,Consistency and Standards,80,0.2177,17.4160
4,Error Prevention,82,0.2529,20.7378
5,Recognition Rather than Recall,85,0.2529,21.4965


And the final usability score is 79.123:

In [13]:
df.RelativeWeight.sum()

79.1233